
# How to use this code
## !Use Google Colab:
https://colab.research.google.com/drive/1cSgfV7vvcuAO7hVk3x-_dDB_3lQp9Rvr
1. Run chunk 1
2. Define the correct file path for the training data in Chunk 2 + run chunk 2
3. Run chunk 3

### To perform neste resampling:
Adapt params in chunk 4 and run it
This stores the best model as a pth file which can be ignored
### To train model given parameters and save model as .pkl:
Adapt paramt in chunk 5 as explained and then run it. The .pkl file is requiered for getting predictions on test data.
### To get model predictions on test data:
In chunk 6 define the path to the .pkl model file, as well as to expression and pData test data sets. Then run the chunk.

In [ ]:
!pip install lifelines
!pip install scikit-learn==1.5.2
!pip install scikit-survival==0.23.1
# Required imports
import numpy as np
import copy
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import torch
from lifelines.utils import concordance_index
from sklearn.utils.validation import check_X_y, check_is_fitted
import logging
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils import check_random_state
from sksurv.util import Surv
import os
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=c9618d2d9ebf95e6943982870fd2d4f551a712494d8dab4a36693b7378afbcc0
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 14.7 MB/s eta 0:00:00


In [ ]:
### Chunk 2
# Defining the pathways to the data used for model training.
# one pData file is needed.
# /content is the folder which serves as the standard upload folder in google colab
CLINICAL_DATA_PATH = '/content/merged_imputed_pData.csv'

In [ ]:
### Chunk 3
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class DeepSurvNet(nn.Module):
    """
    PyTorch based neural network architecture designed for survival prediction.
    This network consists of fully connected layers with ReLU activation,
    dropout for regularization, and a final layer that outputs a single
    hazard prediction value.
    """

    def __init__(self, n_features, hidden_layers=[32, 16], dropout=0.2):
        super().__init__()
        layers = []
        prev_size = n_features
        self.model = None

        # Create network layers
        for size in hidden_layers:
            layers.extend([
                nn.Linear(prev_size, size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_size = size

        # Hazard output layer
        layers.append(nn.Linear(prev_size, 1, bias=False))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class DeepSurvModel(BaseEstimator, RegressorMixin):
    """
    Implementation of the DeepSurv model that integrates
    with scikit-learn, specifying  configurable architecture,
    training procedures, and evaluation metrics.

    The model includes:
    - Customizable neural network architecture
    - Mini-batch training with early stopping
    - CPU/GPU support
    - Concordance index evaluation
    - Compatibility with scikit-learn's cross-validation and pipeline features
    - Reproducible training through seed control

    The model follows scikit-learn's estimator interface by implementing
    fit(), predict(), get_params() and set_params() methods.
    """
    # Main survival model class
    def __init__(self, n_features=None, hidden_layers=[16, 16], dropout=0.5,
                 learning_rate=0.01, device='cpu', random_state=123,
                 batch_size=128, num_epochs=100, patience=15):
        self.n_features = n_features
        self.hidden_layers = hidden_layers
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.device = device if torch.cuda.is_available() and device == 'cuda' else 'cpu'
        self.random_state = random_state
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.patience = patience

        torch.manual_seed(random_state)
        np.random.seed(random_state)

        self.scaler = StandardScaler()
        self.model = None
        self.is_fitted_ = False
        self.training_history_ = {'train_loss': [], 'val_loss': []}
        self.n_features_in_ = None

    def fit(self, X, y):
        # Data validation
        X, y = check_X_y(X, y, accept_sparse=True)

        # Set seeds
        np.random.seed(self.random_state)
        torch.manual_seed(self.random_state)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(self.random_state)
            torch.cuda.manual_seed_all(self.random_state)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        self.n_features_in_ = X.shape[1]
        self.init_network(self.n_features_in_)
        self.model.to(self.device)

        train_loader, val_loader = self._prepare_data(X, y, val_split=0.1)

        # Training loop
        best_val_loss = float('inf')
        best_model_state = None
        counter = 0.0
        for epoch in range(self.num_epochs):
            self.model.train()
            epoch_loss_ = 0.0
            n_batches_ = 0
            for X_batch, time_batch, event_batch in train_loader:
                loss = self._train_step(X_batch, time_batch, event_batch)
                epoch_loss_ += loss
                n_batches_ += 1
            avg_train_loss = epoch_loss_ / n_batches_
            self.training_history_['train_loss'].append(avg_train_loss)

            # Validate
            self.model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for X_batch, time_batch, event_batch in val_loader:
                    val_loss += self._eval_step(X_batch, time_batch, event_batch)

            val_loss = val_loss / len(val_loader)
            self.training_history_['val_loss'].append(val_loss)

            # Model checkpoint
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = copy.deepcopy(self.model.state_dict())
                counter = 0
            else:
                counter += 1

            if counter > self.patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Load best model
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)

        self.is_fitted_ = True
        return self

    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')
        if isinstance(X, pd.DataFrame):
            X = X.values
        X = torch.FloatTensor(X).to(self.device)
        self.model.eval()
        with torch.no_grad():
            risk_scores = self.model(X).cpu().numpy()
        return risk_scores.flatten()

    def score(self, X, y):
        check_is_fitted(self, 'is_fitted_')
        preds = self.predict(X)
        return self.c_index(-preds, y)

    def get_params(self, deep=True):
        return {
            "n_features": self.n_features,
            "hidden_layers": self.hidden_layers,
            "dropout": self.dropout,
            "learning_rate": self.learning_rate,
            "device": self.device,
            "random_state": self.random_state,
            "batch_size": self.batch_size,
            "num_epochs": self.num_epochs,
            "patience": self.patience
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def clone(self):
        super(self).clone()

    def _prepare_data(self, X, y, val_split = 0.1):
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_split, random_state=self.random_state)

        # Process training data
        X_scaled_train = X_train
        times_train = np.ascontiguousarray(y_train['time']).astype(np.float32)
        event_field_train = 'status' if 'status' in y_train.dtype.names else 'event'
        events_train = np.ascontiguousarray(y_train[event_field_train]).astype(np.float32)
        X_tensor_train = torch.FloatTensor(X_scaled_train).to(self.device)
        time_tensor_train = torch.FloatTensor(times_train).to(self.device)
        event_tensor_train = torch.FloatTensor(events_train).to(self.device)

        # Process validation data
        X_scaled_val = X_val
        times_val = np.ascontiguousarray(y_val['time']).astype(np.float32)
        event_field_val = 'status' if 'status' in y_val.dtype.names else 'event'
        events_val = np.ascontiguousarray(y_val[event_field_val]).astype(np.float32)
        X_tensor_val = torch.FloatTensor(X_scaled_val).to(self.device)
        time_tensor_val = torch.FloatTensor(times_val).to(self.device)
        event_tensor_val = torch.FloatTensor(events_val).to(self.device)

        # Create dataloaders
        train_dataset = TensorDataset(X_tensor_train, time_tensor_train, event_tensor_train)
        val_dataset = TensorDataset(X_tensor_val, time_tensor_val, event_tensor_val)

        generator = torch.Generator()
        generator.manual_seed(self.random_state)

        train_loader = DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            generator=generator
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            generator=generator
        )

        return train_loader, val_loader

    def _negative_log_likelihood(self, risk_pred, times, events):
        # Loss calculation
        _, idx = torch.sort(times, descending=True)
        risk_pred = risk_pred[idx]
        events = events[idx]
        log_risk = risk_pred
        risk = torch.exp(log_risk)
        cumsum_risk = torch.cumsum(risk, dim=0)
        log_cumsum_risk = torch.log(cumsum_risk + 1e-10)
        event_loss = events * (log_risk - log_cumsum_risk)
        return -torch.mean(event_loss)

    def _train_step(self, X, times, events):
        # Single training iteration
        self.optimizer.zero_grad()
        risk_pred = self.model(X)
        loss = self._negative_log_likelihood(risk_pred, times, events)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def _eval_step(self, X, times, events):
        # Single evaluation iteration
        risk_pred = self.model(X)
        loss = self._negative_log_likelihood(risk_pred, times, events)
        return loss.item()

    def c_index(self, risk_pred, y):
        # Calculate concordance index
        if not isinstance(y, np.ndarray):
            y = y.detach().cpu().numpy()
        event_field = 'status' if 'status' in y.dtype.names else 'event'
        time = y['time']
        event = y[event_field]
        if not isinstance(risk_pred, np.ndarray):
            risk_pred = risk_pred.detach().cpu().numpy()
        if np.isnan(risk_pred).all():
            return np.nan
        return concordance_index(time, risk_pred, event)

    def init_network(self, n_features):
        # Initialize model and optimizer
        self.model = DeepSurvNet(n_features=n_features, hidden_layers=self.hidden_layers, dropout=self.dropout).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)


        # Helper functions for data handling
def _get_survival_subset(y, indices):
    """Extract survival data subset"""
    subset = np.empty(len(indices), dtype=y.dtype)
    event_field = 'status' if 'status' in y.dtype.names else 'event'
    subset[event_field] = y[event_field][indices]
    subset['time'] = y['time'][indices]
    return subset

def _aggregate_results(results):
    """Process CV results"""
    scores = [res['test_score'] for res in results]
    if np.isnan(scores).all():
        logger.warning(f"Found only NaN values in CV-results: {scores}")
        mean_score, std_score = np.nan, np.nan
    else:
        mean_score = np.nanmean(scores)
        std_score = np.nanstd(scores)

    logger.info(f"Aggregated results:")
    logger.info(f"Mean score: {mean_score:.3f} ± {std_score:.3f}")
    logger.info(f"Individual scores: {scores}")

    return {
        'mean_score': mean_score,
        'std_score': std_score,
        'fold_results': results
    }

def nested_resampling(estimator, X, y, groups, param_grid, monitor = None, ss = GridSearchCV,
                     outer_cv = LeaveOneGroupOut(), inner_cv = LeaveOneGroupOut(), scoring = None):
    """Implementation of the nested resampling logic for hyperparameter optimization"""
    # Main CV implementation
    logger.info("Starting nested resampling...")
    logger.info(f"Data shape: X={X.shape}, groups={len(np.unique(groups))} unique")

    outer_results = []
    splits = list(outer_cv.split(X, y, groups))

    for i, (train_idx, test_idx) in enumerate(splits):
        logger.info(f"\nOuter fold {i+1}")

        fold_seed = 42 + i
        np.random.seed(fold_seed)
        torch.manual_seed(fold_seed)

        X_train = X.iloc[train_idx]
        X_test = X.iloc[test_idx]
        y_train = _get_survival_subset(y, train_idx)
        y_test = _get_survival_subset(y, test_idx)
        train_groups = groups[train_idx] if groups is not None else None

        test_cohort = groups[test_idx][0] if groups is not None else None
        logger.info(f"Test cohort: {test_cohort}")

        inner_gcv = ss(estimator, param_grid, cv = inner_cv, refit = True, n_jobs=1, verbose = 2)
        if monitor is not None:
            inner_results = inner_gcv.fit(X_train, y_train, groups = train_groups, model__monitor = monitor)
        else:
            inner_results = inner_gcv.fit(X_train, y_train, groups = train_groups)

        inner_cv_results = inner_results.cv_results_
        inner_best_params = inner_results.best_params_

        outer_model = inner_results.best_estimator_.named_steps['model']
        test_score = outer_model.score(X_test, y_test)

        logger.info(f"Best parameters: {inner_best_params}")
        logger.info(f"Test score: {test_score:.3f}")

        outer_results.append({
            'test_cohort': test_cohort,
            'test_score': test_score,
            'best_params': inner_best_params,
            'inner_cv_results': inner_cv_results
        })

    return _aggregate_results(outer_results)

class ModellingProcess():
    """
    This class manages the entire modeling process including data preparation,
    nested cross-validation, model training, and result saving. It is a
    standardized way of modeling used for several of the implemented mode types
    and supports both simple training and complex nested resampling approaches.
    Results can be automatically saved and evaluated.
    """
    # Main modeling workflow class
    def __init__(self) -> None:
        self.outer_cv = LeaveOneGroupOut()
        self.inner_cv = LeaveOneGroupOut()
        self.ss = GridSearchCV
        self.pipe = None
        self.cmplt_model = None
        self.cmplt_pipeline = None
        self.nrs = None
        self.X = None
        self.y = None
        self.groups = None
        self.path = None
        self.fname_cv = None

    def prepare_survival_data(self, pdata):
        # Format survival data
        status = pdata['BCR_STATUS'].astype(bool).values
        time = pdata['MONTH_TO_BCR'].astype(float).values
        y = Surv.from_arrays(
            event=status,
            time=time,
            name_event='status',
            name_time='time'
        )
        return y

    def prepare_data(self):
        # Load and process data
        pdata = pd.read_csv(CLINICAL_DATA_PATH, index_col=0)

        feature_cols = ['AGE', 'TISSUE', 'GLEASON_SCORE', 'PRE_OPERATIVE_PSA']
        target_cols = ['MONTH_TO_BCR', 'BCR_STATUS']

        numeric_features = ['AGE', 'GLEASON_SCORE', 'PRE_OPERATIVE_PSA']
        categorical_features = ['TISSUE']

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
            ])

        X_raw = pdata[feature_cols]
        self.X = pd.DataFrame(
            preprocessor.fit_transform(X_raw),
            index=X_raw.index
        )

        target_df = pdata[target_cols]
        self.y = self.prepare_survival_data(target_df)

        self.groups = np.array([idx.split('.')[0] for idx in self.X.index])

    def do_modelling(self, pipeline_steps, config):
        # Execute modeling pipeline
        self._set_seed()

        if config.get("params_mp", None) is not None:
            self.set_params(config['params_mp'])

        if config.get("path", None) is None or config.get("fname_cv", None) is None:
            logger.warning("Didn't get sufficient path info for saving cv-results")
        else:
            self.path = config['path']
            self.fname_cv = config['fname_cv']

        err, mes = self._check_modelling_prerequs(pipeline_steps)
        if err:
            logger.error("Requirements setup error: %s", mes)
            raise Exception(mes)
        else:
            self.pipe = Pipeline(pipeline_steps)

        param_grid, monitor, do_nested_resampling, refit_hp_tuning = self._get_config_vals(config)

        try:
            logger.info("Start model training...")
            logger.info(f"Input data shape: X={self.X.shape}")

            if do_nested_resampling:
                logger.info("Nested resampling...")
                self.nrs = nested_resampling(self.pipe, self.X, self.y, self.groups, param_grid, monitor, self.ss, self.outer_cv, self.inner_cv)
                if (self.fname_cv is not None) and (self.path is not None):
                    self.save_results(self.path, self.fname_cv, model=None, cv_results=self.nrs, pipe=None)
        except Exception as e:
            logger.error(f"Error during nested resampling: {str(e)}")
            raise

        if refit_hp_tuning:
            try:
                logger.info("Do HP Tuning for complete model; refit + set complete model")
                self.fit_cmplt_model(param_grid)
                if (self.fname_cv is not None) and (self.path is not None):
                    self.save_results(self.path, self.fname_cv, model=self.cmplt_model, cv_results=None, pipe=self.cmplt_pipeline)
            except Exception as e:
                logger.error(f"Error during complete model training: {str(e)}")
                raise
        elif refit_hp_tuning is False and do_nested_resampling is False:
            logger.info("Fit complete pipeline wo. HP tuning (on default params)")
            self.cmplt_pipeline = self.pipe.fit(self.X, self.y)
            if (self.fname_cv is not None) and (self.path is not None):
                self.save_results(self.path, self.fname_cv, model=None, cv_results=None, pipe=self.cmplt_pipeline)

        return self.nrs, self.cmplt_model, self.cmplt_pipeline

    def fit_cmplt_model(self, param_grid, monitor=None):
        # Train final model
        logger.info("Do HP Tuning for complete model")
        res = self.ss(
            estimator=self.pipe,
            param_grid=param_grid,
            cv=self.outer_cv,
            n_jobs=1,
            verbose=2,
            refit=True
        )
        if monitor is not None:
            res.fit(self.X, self.y, groups=self.groups, model__monitor=monitor)
        else:
            res.fit(self.X, self.y, groups=self.groups)
        self.resampling_cmplt = res
        self.cmplt_pipeline = res.best_estimator_
        self.cmplt_model = res.best_estimator_.named_steps['model']
        return self.cmplt_model, res

    def save_results(self, path, fname, model=None, cv_results=None, pipe=None):
        # Save model and results
        if model is None:
            logger.warning("Won't save any model, since it's not provided")
        else:
            # Save the model
            model_dir = os.path.join(path, 'model')
            os.makedirs(model_dir, exist_ok=True)
            model.model.to(torch.device('cpu'))
            torch.save(model.model, os.path.join(model_dir, f"{fname}.pth"))
            logger.info(f"Saved model to {model_dir}")

        if cv_results is None:
            logger.warning("Won't save any CV results, since it's not provided")
        else:
            # Save cross-validation results
            results_dir = os.path.join(path, 'results')
            os.makedirs(results_dir, exist_ok=True)
            results_file = os.path.join(results_dir, f"{fname}_cv.csv")
            pd.DataFrame(cv_results).to_csv(results_file)
            logger.info(f"Saved CV results to {results_file}")

    def _check_modelling_prerequs(self, pipeline_steps):
        # Check requirements
        err = False
        mes = ""
        if self.X is None or self.y is None:
            mes = mes + "1) Please call prepare_data() with your preferred config or set X, y, and groups"
            err = True
        if not any('model' in tup for tup in pipeline_steps):
            mes = mes + "2) Caution! Your pipeline must include a step named 'model' for the model"
            err = True
        return err, mes

    def _get_config_vals(self, config):
        # Extract config values
        if config.get("params_cv", None) is None:
            logger.warning("No param grid for (nested) resampling detected - will fit model with default HPs on complete data")
            return None, False, False, False
        if config.get('monitor', None) is None:
            logger.info("No additional monitoring detected")
        return config['params_cv'], config.get('monitor', None), config.get('do_nested_resampling', True), config.get('refit', True)

    def set_params(self, params):
        # Set parameters
        for key, value in params.items():
            setattr(self, key, value)

    def _set_seed(self, seed=1234):
        # Set random seeds
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        global random_state
        random_state = check_random_state(seed)



In [ ]:
### Chunk 4
# Define hyperparameter grid for the nested resampling. To do so, params_cv can be adapted.
# refit and do_nested_resampling should be true
# fname_cv is the name by which the the results are stored in a csv file ->adapt

MODEL_CONFIG = {
    'params_cv'  : {
        # Define the number and size of hidden layers
        'model__hidden_layers': [[512, 128],[512, 256], [256, 128], [1024], [512], [256], [128], [64], [32], [16]],
        # Learning rate for optimization
        'model__learning_rate': [0.00001, 0.0001],
        # Batch size for training
        'model__batch_size': [64],
        # Number of training epochs
        'model__num_epochs': [500],
        # Dropout rate for regularization
        'model__dropout': [0.2, 0.4],
        'model__device': ['cuda']
    },
    'refit': True,
    'do_nested_resampling': True,
    'path' : "",
    'fname_cv' : 'result_intersection'
}


mp = ModellingProcess()
mp.prepare_data()

ds_pipeline_steps = [
    ('model', DeepSurvModel())
]

nstd_res_result, cmplt_model, cmplt_pipeline = mp.do_modelling(ds_pipeline_steps, MODEL_CONFIG)

print(nstd_res_result)

Fitting 8 folds for each of 1 candidates, totalling 8 fits
Early stopping at epoch 45
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   7.3s
Early stopping at epoch 31
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.2s
Early stopping at epoch 38
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.7s
Early stopping at epoch 82
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   3.0s
Early stopping at epoch 34
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, mod

Early stopping at epoch 35
Fitting 9 folds for each of 1 candidates, totalling 9 fits
Early stopping at epoch 17
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   0.6s
Early stopping at epoch 22
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   0.7s
Early stopping at epoch 30
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.3s
Early stopping at epoch 30
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model__learning_rate=0.0001, model__num_epochs=500; total time=   1.5s
Early stopping at epoch 32
[CV] END model__batch_size=64, model__device=cuda, model__dropout=0.2, model__hidden_layers=[16], model

Early stopping at epoch 52
{'mean_score': 0.6299091452259782, 'std_score': 0.07269191686464684, 'fold_results': [{'test_cohort': 'Atlanta_2014_Long', 'test_score': 0.5, 'best_params': {'model__batch_size': 64, 'model__device': 'cuda', 'model__dropout': 0.2, 'model__hidden_layers': [16], 'model__learning_rate': 0.0001, 'model__num_epochs': 500}, 'inner_cv_results': {'mean_fit_time': array([2.25614724]), 'std_fit_time': array([2.03439166]), 'mean_score_time': array([0.00177008]), 'std_score_time': array([0.00069444]), 'param_model__batch_size': masked_array(data=[64],
             mask=[False],
       fill_value=999999), 'param_model__device': masked_array(data=['cuda'],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_model__dropout': masked_array(data=[0.2],
             mask=[False],
       fill_value=1e+20), 'param_model__hidden_layers': masked_array(data=[list([16])],
             mask=[False],
       fill_value='?',
            dtype=object), 'par

In [ ]:
### Chunk 5


# define params for model training
model_params = {
    'hidden_layers': [256, 128],
    'learning_rate': 0.00001,
    'batch_size': 64,
    'num_epochs': 500,
    'dropout': 0.2,
    'device': 'cuda',
    'random_state': 123
}

mp = ModellingProcess()
mp.prepare_data()
# Train model fix given params
model_to_save = DeepSurvModel(**model_params)


model_to_save.fit(mp.X.values, mp.y)

preds_train = model_to_save.predict(mp.X.values)

save_dir = "/content/my_saved_model"
os.makedirs(save_dir, exist_ok=True)
model_file = os.path.join(save_dir, "deep_surv_model_pData.pkl")

with open(model_file, 'wb') as f:
    pickle.dump(model_to_save, f)



Early stopping at epoch 53


In [ ]:
### Chunk 6

# Load model
model_file = "/content/deep_surv_model.pkl"
with open(model_file, 'rb') as f:
    loaded_model = pickle.load(f)
print("Modell erfolgreich geladen")

# Load test data
test_pdata = pd.read_csv('/content/example_exprs.csv', index_col=0)

# Prepare Features
# Numerische Features
numeric_features = ['AGE', 'GLEASON_SCORE', 'PRE_OPERATIVE_PSA']
clinical_numeric = test_pdata[numeric_features]

# Manually create one-hot encoding
clinical_categorical = pd.DataFrame(index=test_pdata.index)
clinical_categorical['TISSUE_Fresh_frozen'] = 1
clinical_categorical['TISSUE_Snap_frozen'] = 0

# Merge all features
X_test = pd.concat([clinical_numeric, clinical_categorical], axis=1)

print("Test Data Shape:", X_test.shape)

# Create survival object
test_status = test_pdata['BCR_STATUS'].astype(bool).values
test_time = test_pdata['MONTH_TO_BCR'].astype(float).values
y_test = Surv.from_arrays(
    event=test_status,
    time=test_time,
    name_event='status',
    name_time='time'
)

X_test_array = X_test.values

# Predicctions on test data
test_predictions = loaded_model.predict(X_test_array)


# Calculate c-index
test_cindex = loaded_model.c_index(-test_predictions, y_test)
print("\nC-index auf Testdaten:", test_cindex)

# save predictions
results_df = pd.DataFrame({
    'sample_id': X_test.index,
    'risk_score': test_predictions
})
results_df.to_csv('/content/test_predictions_clinical.csv')


Modell erfolgreich geladen
Test Data Shape: (1091, 5)

C-index auf Testdaten: 0.6818838670191515
